In [1]:
print('Begins')

Begins


In [2]:
%pwd

'd:\\May-I-Help'

In [3]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import CharacterTextSplitter


d:\ANACONDA\envs\globalenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader

extracter_data = DirectoryLoader(
    "data",
    glob="**/*.txt",
    loader_cls=TextLoader
)

documents = extracter_data.load()


In [5]:
documents

[Document(metadata={'source': 'data\\playmetrics_full_dataset.txt'}, page_content="PLAYMETRICS WEBSITE – EXTRACTED DATASET\n=====================================\n\nTEXT CONTENT\n------------\n\n==============================\nSOURCE URL:\nhttps://www.playmetrics.site/\n==============================\n\nPlayMetrics\n\nMMTT (Multi-Mode Tactical Tracker)\n=================================\n\n1. Overview\n-----------\nMMTT (Multi-Mode Tactical Tracker) is an intelligent tracking and monitoring system designed to collect, analyze, and present tactical or operational data in real time. The system focuses on improving situational awareness, decision-making, and operational efficiency by combining multiple data sources into a unified platform.\n\nMMTT is built to support complex environments where tracking, analysis, and rapid response are critical.\n\n2. Where MMTT Is Used / Present\n-------------------------------\nMMTT is intended for use in environments that require continuous monitoring 

In [6]:
def text_splitter(documents, chunk_size=500, chunk_overlap=100):
    text_splitter = CharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )
    texts = text_splitter.split_documents(documents)
    return texts

In [7]:
text_splitter(documents)

Created a chunk of size 1619, which is longer than the specified 500
Created a chunk of size 716, which is longer than the specified 500


[Document(metadata={'source': 'data\\playmetrics_full_dataset.txt'}, page_content='PLAYMETRICS WEBSITE – EXTRACTED DATASET\n=====================================\n\nTEXT CONTENT\n------------\n\n==============================\nSOURCE URL:\nhttps://www.playmetrics.site/\n==============================\n\nPlayMetrics\n\nMMTT (Multi-Mode Tactical Tracker)\n================================='),
 Document(metadata={'source': 'data\\playmetrics_full_dataset.txt'}, page_content='PlayMetrics\n\nMMTT (Multi-Mode Tactical Tracker)\n=================================\n\n1. Overview\n-----------\nMMTT (Multi-Mode Tactical Tracker) is an intelligent tracking and monitoring system designed to collect, analyze, and present tactical or operational data in real time. The system focuses on improving situational awareness, decision-making, and operational efficiency by combining multiple data sources into a unified platform.'),
 Document(metadata={'source': 'data\\playmetrics_full_dataset.txt'}, page_conte

In [8]:
text_chunks = text_splitter(documents)
print(f"Number of text chunks: {len(text_chunks)}")

Created a chunk of size 1619, which is longer than the specified 500
Created a chunk of size 716, which is longer than the specified 500


Number of text chunks: 63


In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings


In [10]:
def download_model(model_name: str = "sentence-transformers/all-MiniLM-L6-v2"):
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    return embeddings

In [11]:
embeddings=download_model()

C:\Users\Rohit\AppData\Local\Temp\ipykernel_10992\3460608680.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)


In [12]:
querr_Demo=embeddings.embed_query("Hello world")
print(len(querr_Demo))

384


In [13]:
from dotenv import load_dotenv
load_dotenv()


True

In [14]:
import os
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_ENVIRONMENT = os.getenv("PINECONE_ENVIRONMENT")

In [15]:
from pinecone import Pinecone, ServerlessSpec
import os

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"), environment=os.getenv("PINECONE_ENVIRONMENT"))

index_name = "testbook"

existing_indexes = [i.name for i in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

print("Index ready:", index_name)


Index ready: testbook


In [16]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [17]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)

In [18]:
docsearch

In [19]:
retriver = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [20]:
retriver_doc=retriver.invoke("What is MMTT?")

In [21]:
retriver_doc

[Document(id='54c84358-840d-4767-a17d-5b00df445e1a', metadata={'source': 'data\\playmetrics_full_dataset.txt'}, page_content='MMTT is built to support complex environments where tracking, analysis, and rapid response are critical.\n\n\n2. Where MMTT Is Used / Present\n-------------------------------\nMMTT is intended for use in environments that require continuous monitoring and tactical awareness, including:'),
 Document(id='90ecb79e-5fa0-4ffa-8642-207296bf8686', metadata={'source': 'data\\playmetrics_full_dataset.txt'}, page_content='MMTT is built to support complex environments where tracking, analysis, and rapid response are critical.\n\n2. Where MMTT Is Used / Present\n-------------------------------\nMMTT is intended for use in environments that require continuous monitoring and tactical awareness, including:'),
 Document(id='828542d0-9c1c-4308-bccf-d6e5979c2ab0', metadata={'source': 'data\\no_upload.txt'}, page_content='RF SAFETY INFORMATION\n\nMMTT emits radio frequency energy 

In [22]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="gemma2:2b",
    temperature=0.4
)


In [23]:
import langchain
print("LangChain:", langchain.__version__)

import packaging
print("Packaging:", packaging.__version__)


LangChain: 1.2.7
Packaging: 24.2


In [24]:
from langchain_core.prompts import ChatPromptTemplate

rag_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a website assistant. "
        "Answer the user's question using ONLY the information provided in the context. "
        "Do NOT use prior knowledge or make assumptions. "
        "If the answer is not explicitly available in the context, reply exactly with: "
        "'I don’t have enough information from the website to answer that.'"
    ),
    (
        "human",
        "Context:\n{context}\n\nQuestion:\n{question}"
    )
])


In [25]:
import ollama

def ask_groq(context: str, question: str): # Keeping function name for compatibility or rename? Let's rename in logic but if other cells call it...
    # Let's keep ask_groq name but implement with Ollama to avoid breaking later cells
    
    messages = [
        {"role": "system", "content": "You are a website assistant. Answer the user's question using ONLY the information provided in the context. Do NOT use prior knowledge or make assumptions. If the answer is not explicitly available in the context, reply exactly with: 'I don’t have enough information from the website to answer that.'"},
        {"role": "user", "content": f"Context:\n{context}\n\nQuestion:\n{question}"}
    ]

    response = ollama.chat(
        model="gemma2:2b",
        messages=messages,
    )
    return response['message']['content']


In [26]:
import os

DATA_FOLDER = r"D:\May-I-Help\data"   # change if needed

all_text = []

for filename in os.listdir(DATA_FOLDER):
    if filename.endswith(".txt"):
        with open(os.path.join(DATA_FOLDER, filename), "r", encoding="utf-8") as f:
            all_text.append(f.read())

website_context = "\n\n".join(all_text)

print("Context loaded. Length:", len(website_context))


Context loaded. Length: 25295


In [27]:
def ask_groq(context, question):
    messages = [
        {"role": "system", "content": "You are a website assistant. Answer the user's question using ONLY the information provided in the context. Do NOT use prior knowledge or make assumptions. If the answer is not explicitly available in the context, reply exactly with: 'I don’t have enough information from the website to answer that.'"},
        {"role": "user", "content": f"Context:\n{context[:8000]}\n\nQuestion:\n{question}"}
    ]

    response = ollama.chat(
        model="gemma2:2b",
        messages=messages,
    )
    return response['message']['content']


In [28]:
def ask_website(question):
    return ask_groq(website_context, question)


In [29]:
response = ask_website("who is MMTT ?")
print("Response:", response)


Response: MMTT (Multi-Mode Tactical Tracker) is an intelligent tracking and monitoring system designed to collect, analyze, and present tactical or operational data in real time.  It focuses on improving situational awareness, decision-making, and operational efficiency by combining multiple data sources into a unified platform. 



In [30]:
response = ask_website("motive of project ?")
print("Response:", response)


Response: The project aims to address the challenges of fragmented data, slow analysis, and limited situational awareness in tactical operations. MMTT seeks to provide a unified platform that improves decision-making through real-time tracking and analysis of relevant information. 



In [31]:
response = ask_website("what problem did it solve ?")
print("Response:", response)


Response: MMTT (Multi-Mode Tactical Tracker) was built to solve the problems of fragmented data, lack of real-time situational awareness, difficulty in tracking multiple modes or entities simultaneously, and manual analysis being slow and error-prone.  

